In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
from requests import get
import time
import os
import glob

In [2]:
# DEBUG = True
DEBUG = False

In [3]:
interest_rate = 0.0415
ng_tax_discount = 0.375
house_land_value_ratio = 0.85
unit_land_value_ratio = 0.5

In [4]:
def land_tax_calc(land_value):
    if land_value < 250000:
        land_tax = 0

    if land_value >= 250000 and land_value < 600000:
        land_tax = 275 + 0.002 * (land_value - 250000)

    if land_value >= 600000 and land_value < 1000000:
        land_tax = 975 + 0.005 * (land_value - 600000)

    if land_value >= 1000000 and land_value < 1800000:
        land_tax = 2975 + 0.008 * (land_value - 1000000)

    if land_value >= 1800000 and land_value < 3000000:
        land_tax = 9375 + 0.013 * (land_value - 1800000)

    if land_value >= 3000000:
        land_tax = 24975 + 0.0225 * (land_value - 3000000)       
    
    return land_tax

In [5]:
df_suburb = pd.read_csv('australian_postcodes.csv')

In [6]:
df_vic_suburb = df_suburb[df_suburb['state']=='VIC']

In [7]:
df_vic_suburb.head(3)

,postcode,locality,state,long,lat,dc,type
6007,3000,MELBOURNE,VIC,144.956776,-37.817403,CITY DELIVERY CENTRE,Delivery Area
6008,3001,MELBOURNE,VIC,0.000000,0.000000,CITY MAIL PROCESSING CENTRE,Post Office Boxes
6009,3002,EAST MELBOURNE,VIC,144.982207,-37.818517,CITY DELIVERY CENTRE,Delivery Area


In [8]:
df_vic_suburb["locality"] = df_vic_suburb["locality"].apply(lambda x: x.replace('"',''))

C:\Users\Kai\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
df_vic_suburb.loc[:,'locality_dash'] = df_vic_suburb['locality'].apply(lambda x: str(x).replace(' ', '-'))

C:\Users\Kai\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Kai\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
df_vic_suburb['suburb_postcode'] = df_vic_suburb.agg(lambda x: f"{x['locality_dash']}-{x['postcode']}-vic", axis=1)

C:\Users\Kai\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
df_vic_suburb.head(5)

,postcode,locality,state,long,lat,dc,type,locality_dash,suburb_postcode
6007,3000,MELBOURNE,VIC,144.956776,-37.817403,CITY DELIVERY CENTRE,Delivery Area,MELBOURNE,MELBOURNE-3000-vic
6008,3001,MELBOURNE,VIC,0.000000,0.000000,CITY MAIL PROCESSING CENTRE,Post Office Boxes,MELBOURNE,MELBOURNE-3001-vic
6009,3002,EAST MELBOURNE,VIC,144.982207,-37.818517,CITY DELIVERY CENTRE,Delivery Area,EAST-MELBOURNE,EAST-MELBOURNE-3002-vic
6010,3003,WEST MELBOURNE,VIC,144.949592,-37.810871,CITY DELIVERY CENTRE,Delivery Area,WEST-MELBOURNE,WEST-MELBOURNE-3003-vic
6011,3004,MELBOURNE,VIC,144.970161,-37.844246,CITY DELIVERY CENTRE,Delivery Area,MELBOURNE,MELBOURNE-3004-vic


In [12]:
# df_vic_suburb_sample = df_vic_suburb[df_vic_suburb['postcode']==3166]
df_vic_suburb_sample = df_vic_suburb

In [13]:
df_vic_suburb_sample.head(3)

,postcode,locality,state,long,lat,dc,type,locality_dash,suburb_postcode
6007,3000,MELBOURNE,VIC,144.956776,-37.817403,CITY DELIVERY CENTRE,Delivery Area,MELBOURNE,MELBOURNE-3000-vic
6008,3001,MELBOURNE,VIC,0.000000,0.000000,CITY MAIL PROCESSING CENTRE,Post Office Boxes,MELBOURNE,MELBOURNE-3001-vic
6009,3002,EAST MELBOURNE,VIC,144.982207,-37.818517,CITY DELIVERY CENTRE,Delivery Area,EAST-MELBOURNE,EAST-MELBOURNE-3002-vic


In [14]:
df_vic_suburb_analysis_sample = df_vic_suburb_sample[['postcode', 'locality', 'locality_dash', 'suburb_postcode', 'lat', 'long']].copy()

In [15]:
df_vic_suburb_analysis_sample.head(3)

,postcode,locality,locality_dash,suburb_postcode,lat,long
6007,3000,MELBOURNE,MELBOURNE,MELBOURNE-3000-vic,-37.817403,144.956776
6008,3001,MELBOURNE,MELBOURNE,MELBOURNE-3001-vic,0.000000,0.000000
6009,3002,EAST MELBOURNE,EAST-MELBOURNE,EAST-MELBOURNE-3002-vic,-37.818517,144.982207


In [16]:
all_num_bedroom = [2, 3, 4]
all_list_type = ['buy', 'rent']
all_property_type = ['house', 'unit']

for num_bedroom in all_num_bedroom:
    for list_type in all_list_type:
        for property_type in all_property_type:
            if property_type == 'house' and list_type == 'buy':
                df_vic_suburb_analysis_sample['{}-bed-{}-{}'.format(num_bedroom, property_type, list_type)] = np.inf
            
            if property_type == 'unit' and list_type == 'buy':
                df_vic_suburb_analysis_sample['{}-bed-{}-{}'.format((num_bedroom-1), property_type, list_type)] = np.inf
            
            if property_type == 'house' and list_type == 'rent':
                df_vic_suburb_analysis_sample['{}-bed-{}-{}'.format(num_bedroom, property_type, list_type)] = 0
            
            if property_type == 'unit' and list_type == 'rent':
                df_vic_suburb_analysis_sample['{}-bed-{}-{}'.format((num_bedroom-1), property_type, list_type)] = 0

In [17]:
df_vic_suburb_analysis_sample

,postcode,locality,locality_dash,suburb_postcode,lat,long,2-bed-house-buy,1-bed-unit-buy,2-bed-house-rent,1-bed-unit-rent,3-bed-house-buy,2-bed-unit-buy,3-bed-house-rent,2-bed-unit-rent,4-bed-house-buy,3-bed-unit-buy,4-bed-house-rent,3-bed-unit-rent
6007,3000,MELBOURNE,MELBOURNE,MELBOURNE-3000-vic,-37.817403,144.956776,inf,inf,0,0,inf,inf,0,0,inf,inf,0,0
6008,3001,MELBOURNE,MELBOURNE,MELBOURNE-3001-vic,0.000000,0.000000,inf,inf,0,0,inf,inf,0,0,inf,inf,0,0
6009,3002,EAST MELBOURNE,EAST-MELBOURNE,EAST-MELBOURNE-3002-vic,-37.818517,144.982207,inf,inf,0,0,inf,inf,0,0,inf,inf,0,0
6010,3003,WEST MELBOURNE,WEST-MELBOURNE,WEST-MELBOURNE-3003-vic,-37.810871,144.949592,inf,inf,0,0,inf,inf,0,0,inf,inf,0,0
6011,3004,MELBOURNE,MELBOURNE,MELBOURNE-3004-vic,-37.844246,144.970161,inf,inf,0,0,inf,inf,0,0,inf,inf,0,0
6012,3004,ST KILDA ROAD CENTRAL,ST-KILDA-ROAD-CENTRAL,ST-KILDA-ROAD-CENTRAL-3004-vic,-37.844246,144.970161,inf,inf,0,0,inf,inf,0,0,inf,inf,0,0
6013,3004,ST KILDA ROAD MELBOURNE,ST-KILDA-ROAD-MELBOURNE,ST-KILDA-ROAD-MELBOURNE-3004-vic,0.000000,0.000000,inf,inf,0,0,inf,inf,0,0,inf,inf,0,0
6014,3005,WORLD TRADE CENTRE,WORLD-TRADE-CENTRE,WORLD-TRADE-CENTRE-3005-vic,-37.824608,144.950858,inf,inf,0,0,inf,inf,0,0,inf,inf,0,0
6015,3006,SOUTH WHARF,SOUTH-WHARF,SOUTH-WHARF-3006-vic,144.952074,144.952074,inf,inf,0,0,inf,inf,0,0,inf,inf,0,0
6016,3006,SOUTHBANK,SOUTHBANK,SOUTHBANK-3006-vic,0.000000,0.000000,inf,inf,0,0,inf,inf,0,0,inf,inf,0,0


In [18]:
for index, row in df_vic_suburb_analysis_sample.iterrows():
    postcode = row['postcode']
    suburb_dash = row['locality_dash']
    suburb = row['locality']
    suburb_postcode = str(suburb_dash) + '-' + str(postcode) + '-vic'
    if DEBUG:
        print("Retrieve Data for Suburb: {}".format(suburb_postcode))
        
    suburb_profile_file = os.path.join(os.getcwd(), "suburb_profile_REA", "{}.txt".format(suburb_postcode))
    
    if os.path.exists(suburb_profile_file):
        if DEBUG:
            print("{} File Exists".format(suburb_postcode))
        continue
    
    url = 'https://www.realestate.com.au/neighbourhoods/{}'.format(suburb_postcode)
    headers = {"user-agent": "Mizilla/5.0"}
    response = get(url, headers=headers)

    file = open(suburb_profile_file, "w")
    file.write(response.text)
    file.close()

#     break
    
    time.sleep(1)


In [19]:
if DEBUG:
    display(df_vic_suburb_analysis_sample['suburb_postcode'].head(3))

In [20]:
suburb_profile_file_list = glob.glob(os.path.join(os.getcwd(), "suburb_profile_REA/*.txt"))

for suburb_postcode in df_vic_suburb_analysis_sample['suburb_postcode']:
    
    suburb_profile_file = os.path.join(os.getcwd(), "suburb_profile_REA", "{}.txt".format(suburb_postcode))
    
    if DEBUG:
        print("Extract Information from File: '{}'".format(suburb_profile_file))

    if os.path.exists(suburb_profile_file):
        if DEBUG:
            print("{} File Exists".format(suburb_postcode))        
        file = open(suburb_profile_file, "r")
    else:
        if DEBUG:
            print("{} File DOESN'T Exists".format(suburb_postcode))
        continue

    test_txt = file.read()
    html_soup = BeautifulSoup(test_txt, 'html.parser')
    
    if 'page is not found' in str(html_soup):
        if DEBUG:        
            print("PAGE NOT FOUND, SKIP......")
        continue
    
    suburb_postcode = suburb_profile_file.split('\\')[-1].rstrip('.txt')
    
    all_num_bedroom = [2, 3, 4]
    all_list_type = ['buy', 'rent']
    all_property_type = ['house', 'unit']
    
    for num_bedroom in all_num_bedroom:
        for list_type in all_list_type:
            for property_type in all_property_type:
                try:
                    if property_type == 'house':
                        num_bedroom_ajusted = num_bedroom
                        info_list = [i.text for i in html_soup.find_all('a') if list_type in str(i) and 'townhouse-house-with-{}-bedroom'.format(num_bedroom) in str(i)][0].split('$')
                    if property_type == 'unit':
                        num_bedroom_ajusted = num_bedroom-1
                        info_list = [i.text for i in html_soup.find_all('a') if list_type in str(i) and 'unit-apartment-with-{}-bedroom'.format(num_bedroom_ajusted) in str(i)][0].split('$')

                    if len(info_list) == 2:
                        num = info_list[1].replace(',','').rstrip(' PW')
                        if DEBUG:
                            print(" [INFO] {} of {} Bedroom {}: {}".format(list_type, num_bedroom, property_type, num))
                        df_vic_suburb_analysis_sample.loc[df_vic_suburb_analysis_sample['suburb_postcode']==suburb_postcode, '{}-bed-{}-{}'.format(num_bedroom_ajusted, property_type, list_type)] = num
                except:
                    continue

In [21]:
df_vic_suburb_analysis_sample.head(3)

,postcode,locality,locality_dash,suburb_postcode,lat,long,2-bed-house-buy,1-bed-unit-buy,2-bed-house-rent,1-bed-unit-rent,3-bed-house-buy,2-bed-unit-buy,3-bed-house-rent,2-bed-unit-rent,4-bed-house-buy,3-bed-unit-buy,4-bed-house-rent,3-bed-unit-rent
6007,3000,MELBOURNE,MELBOURNE,MELBOURNE-3000-vic,-37.817403,144.956776,inf,337750,640,450,inf,500000,0,610,inf,831500,0,910
6008,3001,MELBOURNE,MELBOURNE,MELBOURNE-3001-vic,0.000000,0.000000,inf,337750,640,450,inf,500000,0,610,inf,831500,0,910
6009,3002,EAST MELBOURNE,EAST-MELBOURNE,EAST-MELBOURNE-3002-vic,-37.818517,144.982207,inf,495000,650,435,inf,748500,923,580,inf,inf,995,950


In [22]:
all_num_bedroom = [2, 3, 4]
all_property_type = ['house', 'unit']

for num_bedroom in all_num_bedroom:
    for property_type in all_property_type:
        if property_type == 'house':
            num_bedroom_ajusted = num_bedroom
        if property_type == 'unit':
            num_bedroom_ajusted = num_bedroom - 1

        df_vic_suburb_analysis_sample['{}-bed-{}-buy'.format(num_bedroom_ajusted, property_type)] = \
          pd.to_numeric(df_vic_suburb_analysis_sample['{}-bed-{}-buy'.format(num_bedroom_ajusted, property_type)])
            
        df_vic_suburb_analysis_sample['{}-bed-{}-rent'.format(num_bedroom_ajusted, property_type)] = \
          pd.to_numeric(df_vic_suburb_analysis_sample['{}-bed-{}-rent'.format(num_bedroom_ajusted, property_type)])
            
            
        df_vic_suburb_analysis_sample['{}-bed-{}-rent-buy-ratio'.format(num_bedroom_ajusted, property_type)] = \
          df_vic_suburb_analysis_sample['{}-bed-{}-rent'.format(num_bedroom_ajusted, property_type)] * 10000 \
            / df_vic_suburb_analysis_sample['{}-bed-{}-buy'.format(num_bedroom_ajusted, property_type)]

        df_vic_suburb_analysis_sample['RANK_{}-bed-{}-rent-buy-ratio'.format(num_bedroom_ajusted, property_type)] = \
          df_vic_suburb_analysis_sample['{}-bed-{}-rent-buy-ratio'.format(num_bedroom_ajusted, property_type)].rank(method='first', ascending=False)

        df_vic_suburb_analysis_sample['INTEREST_{}-bed-{}'.format(num_bedroom_ajusted, property_type)] = \
          - df_vic_suburb_analysis_sample['{}-bed-{}-buy'.format(num_bedroom_ajusted, property_type)] * interest_rate
        
        df_vic_suburb_analysis_sample['Yearly_RENT_{}-bed-{}'.format(num_bedroom_ajusted, property_type)] = \
          df_vic_suburb_analysis_sample['{}-bed-{}-rent'.format(num_bedroom_ajusted, property_type)] * 46 - 3500
        
        df_vic_suburb_analysis_sample['LAND_TAX_{}-bed-{}'.format(num_bedroom_ajusted, property_type)] = \
          - df_vic_suburb_analysis_sample['{}-bed-{}-buy'.format(num_bedroom_ajusted, property_type)].apply(lambda x: land_tax_calc(x * house_land_value_ratio))

        df_vic_suburb_analysis_sample['INVEST_LOSS_{}-bed-{}'.format(num_bedroom_ajusted, property_type)] = \
          (df_vic_suburb_analysis_sample['Yearly_RENT_{}-bed-{}'.format(num_bedroom_ajusted, property_type)] \
             + df_vic_suburb_analysis_sample['INTEREST_{}-bed-{}'.format(num_bedroom_ajusted, property_type)]) * (1 - ng_tax_discount)

        df_vic_suburb_analysis_sample['INVEST_LOSS_RATIO_{}-bed-{}'.format(num_bedroom_ajusted, property_type)] = \
          df_vic_suburb_analysis_sample['INVEST_LOSS_{}-bed-{}'.format(num_bedroom_ajusted, property_type)] \
          / df_vic_suburb_analysis_sample['{}-bed-{}-buy'.format(num_bedroom_ajusted, property_type)] * 1000
        
        df_vic_suburb_analysis_sample['RANK_INVEST_LOSS_{}-bed-{}'.format(num_bedroom_ajusted, property_type)] = \
          df_vic_suburb_analysis_sample['INVEST_LOSS_RATIO_{}-bed-{}'.format(num_bedroom_ajusted, property_type)].rank(method='first', ascending=False)

In [23]:
df_vic_suburb_analysis_sample.to_csv('vic_suburb_analysis.csv',index=False)

In [24]:
num_bedroom = 3
property_type = 'house'

cols = [
    'postcode',
    'locality',
    '{}-bed-{}-buy'.format(num_bedroom, property_type),
    '{}-bed-{}-rent'.format(num_bedroom, property_type),
    'INVEST_LOSS_{}-bed-{}'.format(num_bedroom, property_type),
    'INVEST_LOSS_RATIO_{}-bed-{}'.format(num_bedroom, property_type),
    'RANK_INVEST_LOSS_{}-bed-{}'.format(num_bedroom, property_type),   
]

In [25]:
top_n = 50

df_vic_suburb_analysis_sample[df_vic_suburb_analysis_sample['RANK_INVEST_LOSS_{}-bed-{}'.format(num_bedroom, property_type)] <= top_n][cols]\
  .sort_values(by = 'RANK_INVEST_LOSS_{}-bed-{}'.format(num_bedroom, property_type))

,postcode,locality,3-bed-house-buy,3-bed-house-rent,INVEST_LOSS_3-bed-house,INVEST_LOSS_RATIO_3-bed-house,RANK_INVEST_LOSS_3-bed-house
9353,3943,SORRENTO,1145000.0,1145000,3.288686e+07,28722.152020,1.0
7572,3458,TRENTHAM,605000.0,605000,1.737587e+07,28720.446798,2.0
8599,3707,CORRYONG,156500.0,250,9.407812e+02,6.011382,3.0
6722,3264,TERANG,182500.0,270,8.414062e+02,4.610445,4.0
6851,3300,HAMILTON,200000.0,280,6.750000e+02,3.375000,5.0
7748,3480,DONALD,143000.0,220,4.284375e+02,2.996066,6.0
9221,3888,ORBOST,190000.0,265,5.031250e+02,2.648026,7.0
7289,3380,STAWELL,185000.0,260,4.890625e+02,2.643581,8.0
7470,3418,NHILL,155500.0,230,3.917188e+02,2.519092,9.0
7250,3377,ARARAT,212000.0,280,3.637500e+02,1.715802,10.0


In [26]:
select_suburb = [
    'BERWICK',
    'HUGHESDALE',
    'GLEN IRIS'
]

df_vic_suburb_analysis_sample[df_vic_suburb_analysis_sample['locality'].isin(select_suburb)][cols]\
  .sort_values(by = 'RANK_INVEST_LOSS_{}-bed-{}'.format(num_bedroom, property_type))

,postcode,locality,3-bed-house-buy,3-bed-house-rent,INVEST_LOSS_3-bed-house,INVEST_LOSS_RATIO_3-bed-house,RANK_INVEST_LOSS_3-bed-house
8849,3806,BERWICK,610000.0,380,-7084.37500,-11.613730,295.0
6407,3166,HUGHESDALE,1038500.0,490,-15036.09375,-14.478665,449.0
6359,3146,GLEN IRIS,1630000.0,635,-26209.37500,-16.079371,482.0
